#Installation

In [ ]:
! pip install flair

In [ ]:
!pip install 'flair==0.10'

#Train

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

In [ ]:
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '/content/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train(Nsurl).txt',
                             test_file = 'A(space to _).txt',
                            # dev_file = 'dev split 2.txt'
                              )

2023-01-05 09:01:33,427 Reading data from /content
2023-01-05 09:01:33,441 Train: /content/train(Nsurl).txt
2023-01-05 09:01:33,442 Dev: None
2023-01-05 09:01:33,445 Test: /content/A(space to _).txt


In [ ]:
# tag to predict
tag_type = 'ner'
# make tag dictionary from the corpus
tag_dictionaryy = corpus.make_tag_dictionary(tag_type=tag_type)

<ipython-input-6-147f0cc9863b>:4: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionaryy = corpus.make_tag_dictionary(tag_type=tag_type)


In [ ]:
'''from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, TokenEmbeddings'''
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

'''from typing import List
embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)'''

#xlm-roberta-large
embeddings = TransformerWordEmbeddings(model='HooshvareLab/bert-base-parsbert-uncased',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

2023-01-05 09:02:24,609 No model_max_length in Tokenizer's config.json - setting it to 512. Specify desired model_max_length by passing it as attribute to embedding instance.


Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

In [ ]:
from flair.models import SequenceTagger
'''tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)'''
tagger = SequenceTagger(hidden_size=512,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionaryy,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )
print(tagger)

In [ ]:
from flair.trainers import ModelTrainer
'''trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)'''

trainer = ModelTrainer(tagger, corpus)
#resources/taggers/sota-ner-flert
trainer.fine_tune('resources/taggers/example-ner',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  shuffle=True
                  )


2023-01-05 09:05:30,442 ----------------------------------------------------------------------------------------------------
2023-01-05 09:05:30,454 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(100000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): 

Token indices sequence length is longer than the specified maximum sequence length for this model (582 > 512). Running this sequence through the model will result in indexing errors


2023-01-05 09:18:35,381 epoch 1 - iter 1956/6529 - loss 1.34442162 - samples/sec: 10.26 - lr: 0.000001
2023-01-05 09:22:53,026 epoch 1 - iter 2608/6529 - loss 1.10877273 - samples/sec: 10.13 - lr: 0.000002
2023-01-05 09:27:16,922 epoch 1 - iter 3260/6529 - loss 0.95119937 - samples/sec: 9.89 - lr: 0.000002
2023-01-05 09:31:40,011 epoch 1 - iter 3912/6529 - loss 0.84310935 - samples/sec: 9.92 - lr: 0.000003
2023-01-05 09:35:56,128 epoch 1 - iter 4564/6529 - loss 0.76310926 - samples/sec: 10.19 - lr: 0.000003
2023-01-05 09:40:02,255 epoch 1 - iter 5216/6529 - loss 0.70852212 - samples/sec: 10.60 - lr: 0.000004
2023-01-05 09:44:16,031 epoch 1 - iter 5868/6529 - loss 0.65900818 - samples/sec: 10.28 - lr: 0.000004
2023-01-05 09:48:19,840 epoch 1 - iter 6520/6529 - loss 0.61608114 - samples/sec: 10.70 - lr: 0.000005
2023-01-05 09:48:23,050 ----------------------------------------------------------------------------------------------------
2023-01-05 09:48:23,052 EPOCH 1 done: loss 0.6157 - l

{'test_score': 0.8739376770538244,
 'dev_score_history': [0.7560521415270018],
 'train_loss_history': [0.6156570356755657],
 'dev_loss_history': [tensor(0.1093, device='cuda:0')]}

#Train(gold)

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

In [ ]:
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '/content/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'ner_dataset_final.txt',
                              #test_file = 'test_gold.txt',
                              #dev_file = 'dev split 2.txt'
                              )

2022-12-19 18:58:39,471 Reading data from /content
2022-12-19 18:58:39,474 Train: /content/ner_dataset_final.txt
2022-12-19 18:58:39,476 Dev: None
2022-12-19 18:58:39,480 Test: None


UnicodeDecodeError: ignored

In [ ]:
# tag to predict
tag_type = 'ner'
# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

2022-12-19 19:00:42,728 Computing label dictionary. Progress:


100%|██████████| 11578/11578 [00:00<00:00, 12960.67it/s]

2022-12-19 19:00:43,660 Corpus contains the labels: ner (#469888)
2022-12-19 19:00:43,663 Created (for label 'ner') Dictionary with 44 tags: <unk>, O, B-RELIGIOUS_CONCEPT, B-CARDINAL, B-PRODUCT, B-RELIGIOUS_ACT, B-PERSON, B-NORP, I-PRODUCT, I-NORP, B-WORK_OF_ART, B-ORDINAL, I-PERSON, I-RELIGIOUS_CONCEPT, B-GPE, I-WORK_OF_ART, B-EVENT, I-CARDINAL, B-ORGANIZATION, I-ORGANIZATION, B-LAW, I-LAW, B-PERCENT, I-PERCENT, B-DATE, I-EVENT, I-GPE, I-DATE, B-LOCATION, I-LOCATION, B-LANGUAGE, I-LANGUAGE, B-RELIGIOUS_TEXT, I-RELIGIOUS_TEXT, I-RELIGIOUS_ACT, B-FACILITY, I-FACILITY, B-QUALITY, B-MONEY, I-MONEY, B-TIME, I-TIME, I-ORDINAL, I-QUALITY


In [ ]:
'''from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings, TokenEmbeddings'''
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

'''from typing import List
embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)'''

#xlm-roberta-large
embeddings = TransformerWordEmbeddings(model='HooshvareLab/bert-base-parsbert-uncased',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

2022-11-08 08:52:28,481 No model_max_length in Tokenizer's config.json - setting it to 512. Specify desired model_max_length by passing it as attribute to embedding instance.


Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

In [ ]:
from flair.models import SequenceTagger
'''tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)'''
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )
print(tagger)

SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(100000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias

In [ ]:
from flair.trainers import ModelTrainer
'''trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)'''

trainer = ModelTrainer(tagger, corpus)
#resources/taggers/sota-ner-flert
trainer.fine_tune('resources/taggers/example-ner',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )


2022-11-08 08:52:54,715 ----------------------------------------------------------------------------------------------------
2022-11-08 08:52:54,721 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(100000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): 

Token indices sequence length is longer than the specified maximum sequence length for this model (593 > 512). Running this sequence through the model will result in indexing errors


2022-11-08 08:55:34,767 epoch 1 - iter 358/3588 - loss 3.44634254 - samples/sec: 8.95 - lr: 0.000000
2022-11-08 08:58:09,917 epoch 1 - iter 716/3588 - loss 2.30693976 - samples/sec: 9.23 - lr: 0.000001
2022-11-08 09:00:45,886 epoch 1 - iter 1074/3588 - loss 1.81606570 - samples/sec: 9.18 - lr: 0.000001
2022-11-08 09:03:16,267 epoch 1 - iter 1432/3588 - loss 1.51976781 - samples/sec: 9.52 - lr: 0.000002
2022-11-08 09:05:58,578 epoch 1 - iter 1790/3588 - loss 1.32636627 - samples/sec: 8.82 - lr: 0.000002
2022-11-08 09:08:26,613 epoch 1 - iter 2148/3588 - loss 1.19646948 - samples/sec: 9.68 - lr: 0.000003
2022-11-08 09:10:58,902 epoch 1 - iter 2506/3588 - loss 1.08879287 - samples/sec: 9.41 - lr: 0.000003
2022-11-08 09:13:29,360 epoch 1 - iter 2864/3588 - loss 1.01410097 - samples/sec: 9.52 - lr: 0.000004
2022-11-08 09:15:56,984 epoch 1 - iter 3222/3588 - loss 0.94656684 - samples/sec: 9.70 - lr: 0.000004
2022-11-08 09:18:27,102 epoch 1 - iter 3580/3588 - loss 0.89014388 - samples/sec: 9.

RuntimeError: ignored

#Run

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')
# create example sentence
sentence = Sentence('I love Berlin')
# predict the tags
model.predict(sentence)
print(sentence.to_tagged_string())

In [ ]:
from flair.datasets import DataLoader, CONLL_03

# load the model you want to evaluate
tagger: SequenceTagger = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

# load the dataset
#corpus = CONLL_03(in_memory=True).downsample(0.01)
columns = {0 : 'text', 1 : '' , 2 : '' , 3 : 'ner'}
# directory where the data resides
data_folder = '/content/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'valid.txt')

#DataLoader(corpus.test, batch_size=1)
# run evaluation procedure
result, score = tagger.evaluate(corpus.test, mini_batch_size=1, gold_label_type='ner' , out_path=f"predictions2.txt")
print(result.log_line)

In [ ]:
result, score = tagger.evaluate(corpus.test, mini_batch_size=1, gold_label_type='ner' , out_path=f"predictions2.txt")
print(result.log_line)

In [ ]:
result, score = tagger.evaluate(corpus.test, gold_label_type='ner' , out_path=f"predictions.txt")
print(result.detailed_results)